In [1]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=fbf7545358816c1b36a4f8259a26a7ebd9baa3c386cd954e9b569b8fdec97f7a
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


## Import necessary libraries

In [2]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import pandas as pd

In [3]:
# Constants
MODEL_NAME = "gpt2"
BATCH_SIZE = 1
EPOCHS = 1
PROMPT_TOKEN = "[SUMMARIZE]"
MAX_LEN = 1024
LEARNING_RATE = 1e-4

In [4]:
# Soft Prompt Vocabulary
soft_prompt_vocab = ["[SUMMARIZE]"]  # Define your custom vocabulary here

# Create a word2idx dictionary for the soft prompt vocabulary
soft_prompt_word2idx = {word: idx for idx, word in enumerate(soft_prompt_vocab)}

num_prompts = len([soft_prompt_word2idx[word] for word in PROMPT_TOKEN.split()])
prompt_id = torch.tensor([soft_prompt_word2idx[word] for word in PROMPT_TOKEN.split()])

In [5]:
print(num_prompts)

1


## Loading Soft Prompt GPT2 model

In [6]:
# Model Architecture
class GPT2WithSoftPrompt(torch.nn.Module):
    def __init__(self, model_name, num_prompts, embedding_size=768):
        super().__init__()
        self.gpt2 = GPT2LMHeadModel.from_pretrained(model_name)
        self.soft_prompt = torch.nn.Embedding(num_prompts, embedding_size)

    def forward(self, input_ids, prompt_ids):
        prompt_embeddings = self.soft_prompt(prompt_ids)
        base_embeddings = self.gpt2.transformer.wte(input_ids)
        embeddings = torch.cat([prompt_embeddings, base_embeddings.squeeze(0)], dim=0)
        outputs = self.gpt2(inputs_embeds=embeddings)
        return outputs

## Preprocess Data

In [7]:
# Data Loading and Preprocessing
def load_and_preprocess_data(file_path, num_prompts):
    df = pd.read_csv(file_path)
    df = df.dropna().sample(frac=0.1)  # Use only 10% of the data

    # Perform preprocessing on the data
    tokenized_articles = []
    tokenized_summaries = []

    tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)

    for article, summary in zip(df["article"], df["highlights"]):
        # Adjust the maximum length of articles to avoid exceeding MAX_LEN
        max_length_article = MAX_LEN - num_prompts
        article_tokens = tokenizer.encode(article, truncation=True, max_length=max_length_article)
        summary_tokens = tokenizer.encode(summary, truncation=True, max_length=300)

        max_length_summary = MAX_LEN
        padded_article = article_tokens + [tokenizer.eos_token_id] * (max_length_article - len(article_tokens))
        padded_summary = summary_tokens + [tokenizer.eos_token_id] * (max_length_summary - len(summary_tokens))

        tokenized_articles.append(padded_article)
        tokenized_summaries.append(padded_summary)


    return tokenized_articles, tokenized_summaries

In [8]:
# Load and preprocess the data
tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)

tokenized_articles_train, tokenized_summaries_train = load_and_preprocess_data("/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv", num_prompts)
tokenized_articles_validation, tokenized_summaries_validation = load_and_preprocess_data("/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/validation.csv", num_prompts)
tokenized_articles_test, tokenized_summaries_test = load_and_preprocess_data("/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/test.csv", num_prompts)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in 

In [12]:
tokenized_articles_train = tokenized_articles_train[:21000]
tokenized_summaries_train = tokenized_summaries_train[:21000]

tokenized_articles_validation = tokenized_articles_validation[:6000]
tokenized_summaries_validation = tokenized_summaries_validation[:6000]

tokenized_articles_test = tokenized_articles_test[:3000]
tokenized_summaries_test = tokenized_summaries_test[:3000]

In [13]:
model_vanilla = GPT2LMHeadModel.from_pretrained(MODEL_NAME)

In [14]:
# # Model Initialization
model = GPT2WithSoftPrompt(MODEL_NAME, num_prompts).to(device)

In [15]:
from tqdm import tqdm

BATCH_SIZE = 1
EPOCHS = 1
GRADIENT_ACCUMULATION_STEPS = 4
GRADIENT_CLIP_NORM = 1.0
EARLY_STOPPING_PATIENCE = 1
prompt_id = prompt_id.to(device)

# Import cross_entropy_loss
from torch.nn import CrossEntropyLoss

## Rouge Score Calculation

In [18]:
def calculate_rouge(predictions, references):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = [scorer.score(pred, ref) for pred, ref in zip(predictions, references)]
    avg_rouge1 = sum([s['rouge1'].fmeasure for s in scores]) / len(scores)
    avg_rouge2 = sum([s['rouge2'].fmeasure for s in scores]) / len(scores)
    avg_rougeL = sum([s['rougeL'].fmeasure for s in scores]) / len(scores)
    return avg_rouge1, avg_rouge2, avg_rougeL

In [14]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

old_params = count_parameters(model_vanilla)
new_params = count_parameters(model)
added_params = new_params - old_params
print("Trainable params: ", added_params)

Trainable params:  768


## Training

In [15]:
import time
import torch
from tqdm import tqdm
from torch.nn import CrossEntropyLoss
from rouge_score import rouge_scorer

def fine_tune_on_summarization(model, train_articles, train_summaries, val_articles, val_summaries):
    optimizer = torch.optim.Adam(model.soft_prompt.parameters(), LEARNING_RATE)
    criterion = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.eos_token_id)
    best_val_loss = float('inf')
    no_improvement_epochs = 0

    total_start_time = time.time()  # Start measuring total training time

    for epoch in range(EPOCHS):
        model.train()
        train_loss = 0

        # GPU time measurement
        train_start_event = torch.cuda.Event(enable_timing=True)
        train_end_event = torch.cuda.Event(enable_timing=True)
        train_start_event.record()  # Start recording GPU time

        with tqdm(enumerate(zip(train_articles, train_summaries)), total=len(train_articles), desc=f"Epoch {epoch + 1}/{EPOCHS}", unit="batch") as progress:
            for idx, (article, summary) in progress:
                input_ids = torch.tensor(article).to(device)
                labels = torch.tensor(summary).to(device)

                outputs = model(input_ids, prompt_id)
                logits = outputs.logits

                # Calculate loss using predefined criterion
                loss = criterion(logits.view(-1, logits.size(-1)), labels.view(-1))
                train_loss += loss.item()

                # Backpropagation
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), GRADIENT_CLIP_NORM)
                optimizer.step()
                optimizer.zero_grad()

            avg_train_loss = train_loss / len(train_articles)
            print(f"Train Loss (Epoch {epoch + 1}): {avg_train_loss:.4f}")

        train_end_event.record()  # End recording GPU time
        train_end_event.synchronize()  # Wait for the events to finish
        train_gpu_time = train_start_event.elapsed_time(train_end_event)  # Time in milliseconds

        # Validation
        model.eval()
        val_loss = 0
        val_predictions = []
        val_references = []

        # GPU time measurement for validation
        val_start_event = torch.cuda.Event(enable_timing=True)
        val_end_event = torch.cuda.Event(enable_timing=True)
        val_start_event.record()  # Start recording GPU time

        with torch.no_grad():
            for article, summary in tqdm(zip(val_articles, val_summaries), total=len(val_articles), desc="Validation", unit="batch"):
                input_ids = torch.tensor(article).to(device)
                labels = torch.tensor(summary).to(device)

                outputs = model(input_ids, prompt_id)

                # Calculate validation loss
                ignore_index = tokenizer.eos_token_id if tokenizer.eos_token_id is not None else -100
                loss = CrossEntropyLoss(ignore_index=ignore_index)(outputs.logits.view(-1, outputs.logits.size(-1)), labels.view(-1))
                val_loss += loss.item()

                # Decode predictions and add to ROUGE computation lists
                predicted_token_ids = torch.argmax(outputs.logits, dim=-1)
                pred_text = tokenizer.decode(predicted_token_ids.squeeze(0), skip_special_tokens=True)
                ref_text = tokenizer.decode(labels, skip_special_tokens=True)
                val_predictions.append(pred_text)
                val_references.append(ref_text)

            avg_val_loss = val_loss / len(val_articles)
            avg_rouge1, avg_rouge2, avg_rougeL = calculate_rouge(val_predictions, val_references)
            print(f"Val Loss (Epoch {epoch + 1}): {avg_val_loss:.4f}")
            print(f"Val ROUGE-1: {avg_rouge1:.4f}, Val ROUGE-2: {avg_rouge2:.4f}, Val ROUGE-L: {avg_rougeL:.4f}")

        val_end_event.record()  # End recording GPU time
        val_end_event.synchronize()  # Wait for the events to finish
        val_gpu_time = val_start_event.elapsed_time(val_end_event)  # Time in milliseconds

        # Early stopping
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            no_improvement_epochs = 0
        else:
            no_improvement_epochs += 1
            if no_improvement_epochs >= EARLY_STOPPING_PATIENCE:
                print(f"Early stopping after {EARLY_STOPPING_PATIENCE} epochs without improvement.")
                break

        # Print GPU compute time for the epoch
        print(f"GPU Compute Time (Train Epoch {epoch + 1}): {train_gpu_time:.2f} ms")
        print(f"GPU Compute Time (Validation Epoch {epoch + 1}): {val_gpu_time:.2f} ms")
        
        torch.save(model.state_dict(), f"prompt_tuning_{epoch}.pth")

    total_end_time = time.time()  # End measuring total training time
    total_training_time = total_end_time - total_start_time
    print(f"Total training time: {total_training_time:.2f} seconds")

    return model


In [16]:
import time

In [1]:
fine_tuned_model = fine_tune_on_summarization(model, tokenized_articles_train, tokenized_summaries_train, tokenized_articles_validation, tokenized_summaries_validation)

torch.save(fine_tuned_model.state_dict(), 'prompt_tuning.pth')

Epoch 1/1: 100%|██████████| 21000/21000 [1:36:58<00:00,  3.61batch/s]
Train Loss (Epoch 1): 9.4259
Validation: 100%|██████████| 1337/1337 [02:30<00:00,  8.87batch/s]
Val Loss (Epoch 1): 8.8849
Val ROUGE-1: 0.0912, Val ROUGE-2: 0.0229, Val ROUGE-L: 0.0579
GPU Compute Time (Train Epoch 1): 5819270.50 ms
GPU Compute Time (Validation Epoch 1): 195395.25 ms
Total training time: 5818 seconds


## Evaluation

In [13]:
fine_tuned_model.eval()
test_loss=0.0
test_predictions = []
test_references = []

with torch.no_grad():
    for article, summary in tqdm(zip(tokenized_articles_test, tokenized_summaries_test), total=len(tokenized_articles_test), desc="Testing", unit="batch"):
        input_ids = torch.tensor(article).to(device)
        labels = torch.tensor(summary).to(device)
        outputs = fine_tuned_model(input_ids, prompt_id)

        # Calculate loss manually
        ignore_index = tokenizer.eos_token_id if tokenizer.eos_token_id is not None else -100
        loss = CrossEntropyLoss(ignore_index=ignore_index)(outputs.logits.view(-1, outputs.logits.size(-1)), labels.view(-1))
        test_loss += loss.item()

        predicted_token_ids = torch.argmax(outputs.logits, dim=-1)
        pred_text = tokenizer.decode(predicted_token_ids.squeeze(0), skip_special_tokens=True)
        ref_text = tokenizer.decode(labels, skip_special_tokens=True)
        test_predictions.append(pred_text)
        test_references.append(ref_text)

    # Calculate test ROUGE scores
    avg_rouge1_test, avg_rouge2_test, avg_rougeL_test = calculate_rouge(test_predictions, test_references)
    print(f"Test ROUGE-1: {avg_rouge1_test:.4f}, Test ROUGE-2: {avg_rouge2_test:.4f}, Test ROUGE-L: {avg_rougeL_test:.4f}")

Testing: 100%|██████████| 1149/1149 [02:07<00:00,  9.01batch/s]
Test ROUGE-1: 0.0891, Test ROUGE-2: 0.0221, Test ROUGE-L: 0.0566


In [16]:
# Initialize a new instance of the model
model = GPT2WithSoftPrompt(MODEL_NAME, num_prompts).to(device)

# Load the saved model state_dict
model.load_state_dict(torch.load('prompt_tuning.pth'))


/tmp/ipykernel_30/2460819413.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/kaggle/input/prompttuning/pytorch/default/1/prompt_tuning

<All keys matched successfully>

In [21]:
model.eval()
test_loss = 0
test_predictions = []
test_references = []
test_start_event = torch.cuda.Event(enable_timing=True)
test_end_event = torch.cuda.Event(enable_timing=True)
test_start_event.record()

with torch.no_grad():
    for article, summary in tqdm(zip(tokenized_articles_test, tokenized_summaries_test), total=len(tokenized_articles_test), desc="Testing", unit="batch"):
        input_ids = torch.tensor(article).to(device)
        labels = torch.tensor(summary).to(device)
        outputs = model(input_ids, prompt_id)

        # Calculate loss manually
        ignore_index = tokenizer.eos_token_id if tokenizer.eos_token_id is not None else -100
        loss = CrossEntropyLoss(ignore_index=ignore_index)(outputs.logits.view(-1, outputs.logits.size(-1)), labels.view(-1))
        test_loss += loss.item()

        predicted_token_ids = torch.argmax(outputs.logits, dim=-1)
        pred_text = tokenizer.decode(predicted_token_ids.squeeze(0), skip_special_tokens=True)
        ref_text = tokenizer.decode(labels, skip_special_tokens=True)
        test_predictions.append(pred_text)
        test_references.append(ref_text)

    # Calculate test ROUGE scores
    avg_rouge1_test, avg_rouge2_test, avg_rougeL_test = calculate_rouge(test_predictions, test_references)
    print(f"Test ROUGE-1: {avg_rouge1_test:.4f}, Test ROUGE-2: {avg_rouge2_test:.4f}, Test ROUGE-L: {avg_rougeL_test:.4f}")

Testing: 100%|██████████| 1149/1149 [02:07<00:00,  9.01batch/s]


Test ROUGE-1: 0.0891, Test ROUGE-2: 0.0221, Test ROUGE-L: 0.0566
